In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import pymongo
import pdb
from datetime import datetime, timedelta
from dateutil.relativedelta import *
from scipy.io import loadmat
import os
import glob

In [2]:
def transform_lon(lon):
    '''
    Transforms longitude from absolute to -180 to 180 deg
    '''
    if lon >= 180:
        lon -= 360
    return lon

def make_doc(df, date, presLevel, dataVariable, param, measurement, gridName, units):
    '''
    Takes df and converts it into a document for mongodb
    '''
    doc = {}
    df = df.rename(index=str, columns={dataVariable: 'value'})
    dataDict = df.to_dict(orient='records')
    print(gridName)
    doc['gridName'] = gridName
    doc['measurement'] = measurement #temperature or psal
    doc['units'] = units # degrees celsius or psu
    doc['param'] = param # anomaly or mean
    doc['data'] = dataDict 
    doc['variable'] = dataVariable # ARGO_TEMPERATURE_ANOMALY or ARGO_TEMPERATURE_MEAN
    doc['date'] = date
    doc['pres'] = float(presLevel)
    doc['cellsize'] = 1  #  Degree
    doc['NODATA_value'] = np.NaN
    return doc

def insert_pres_time_grid(xrDataArray, coll, param, measurement, gridName, units,
                          dataVariable, insertOne=False):
    for tdx, chunk in xrDataArray.groupby('TIME'):
        month = int(tdx % 12 + 1)
        year = int(2004 + tdx // 12)
        date = datetime.strptime('{0}-{1}'.format(year, month), '%Y-%m')
        if not year == 2007: # only add 2007
            continue
        print(date)
        df = chunk.to_dataframe()
        df = df.reset_index()
        df = df.rename(columns={'LATITUDE':'lat', 'LONGITUDE':'lon'})
        df['lon'] = df['lon'].apply(lambda lon: transform_lon(lon))

        for pdx, presDf in df.groupby('PRESSURE'):
            if not pdx in [10, 50, 200]:
                continue
            presDf = presDf.drop(['TIME', 'PRESSURE'], axis=1)
            doc = make_doc(presDf, date, pdx, dataVariable, param, measurement, gridName, units)
            if insertOne: # Use for testing
                coll.insert_one(doc)
                return
            else:
                coll.insert_one(doc)

In [3]:
def create_collection(dbName, collectionName):
    dbUrl = 'mongodb://localhost:27017/'
    client = pymongo.MongoClient(dbUrl)
    db = client[dbName]
    coll = db[collectionName]
    coll = init_profiles_collection(coll)
    return coll

def init_profiles_collection(coll):
    try:
        coll.create_index([('date', pymongo.DESCENDING)])
        coll.create_index([('pres', pymongo.DESCENDING)])
        coll.create_index([('data.lat', pymongo.DESCENDING)])
        coll.create_index([('data.lon', pymongo.ASCENDING)])
        
        #may want to store as geojson feature collection one day
        #coll.create_index([('data.geometries', pymongo.GEOSPHERE)])

    except:
        logging.warning('not able to get collections or set indexes')
    return coll

In [4]:
rgFilename = '/home/tyler/Desktop/RG_ArgoClim_Temp.nc'
rg = xr.open_dataset(rgFilename, decode_times=False)

In [5]:
rg.info()

xarray.Dataset {
dimensions:
	LATITUDE = 145 ;
	LONGITUDE = 360 ;
	PRESSURE = 58 ;
	TIME = 181 ;
	bnds = 2 ;

variables:
	float32 LONGITUDE(LONGITUDE) ;
		LONGITUDE:units = degrees_east ;
		LONGITUDE:modulo = 360.0 ;
		LONGITUDE:point_spacing = even ;
		LONGITUDE:axis = X ;
		LONGITUDE:standard_name = longitude ;
	float32 LATITUDE(LATITUDE) ;
		LATITUDE:units = degrees_north ;
		LATITUDE:point_spacing = even ;
		LATITUDE:axis = Y ;
		LATITUDE:standard_name = latitude ;
	float32 PRESSURE(PRESSURE) ;
		PRESSURE:units = dbar ;
		PRESSURE:positive = down ;
		PRESSURE:point_spacing = uneven ;
		PRESSURE:axis = Z ;
		PRESSURE:standard_name = depth ;
		PRESSURE:bounds = PRESSURE_bnds ;
	float32 PRESSURE_bnds(PRESSURE, bnds) ;
	float32 TIME(TIME) ;
		TIME:units = months since 2004-01-01 00:00:00 ;
		TIME:time_origin = 01-JAN-2004 00:00:00 ;
		TIME:axis = T ;
		TIME:standard_name = time ;
	float32 ARGO_TEMPERATURE_ANOMALY(TIME, PRESSURE, LATITUDE, LONGITUDE) ;
		ARGO_TEMPERATURE_ANOMALY:units =

In [6]:
bnds = rg['PRESSURE_bnds']

In [7]:
bnds

<xarray.DataArray 'PRESSURE_bnds' (PRESSURE: 58, bnds: 2)>
array([[-1.25000e+00,  6.25000e+00],
       [ 6.25000e+00,  1.50000e+01],
       [ 1.50000e+01,  2.50000e+01],
       [ 2.50000e+01,  3.50000e+01],
       [ 3.50000e+01,  4.50000e+01],
       [ 4.50000e+01,  5.50000e+01],
       [ 5.50000e+01,  6.50000e+01],
       [ 6.50000e+01,  7.50000e+01],
       [ 7.50000e+01,  8.50000e+01],
       [ 8.50000e+01,  9.50000e+01],
       [ 9.50000e+01,  1.05000e+02],
       [ 1.05000e+02,  1.15000e+02],
       [ 1.15000e+02,  1.25000e+02],
       [ 1.25000e+02,  1.35000e+02],
       [ 1.35000e+02,  1.45000e+02],
       [ 1.45000e+02,  1.55000e+02],
       [ 1.55000e+02,  1.65000e+02],
       [ 1.65000e+02,  1.76250e+02],
       [ 1.76250e+02,  1.91250e+02],
       [ 1.91250e+02,  2.10000e+02],
       [ 2.10000e+02,  2.30000e+02],
       [ 2.30000e+02,  2.50000e+02],
       [ 2.50000e+02,  2.70000e+02],
       [ 2.70000e+02,  2.90000e+02],
       [ 2.90000e+02,  3.10000e+02],
       [ 3.10000

In [17]:
rgFilename = '/home/tyler/Desktop/RG_ArgoClim_Temp.nc'
rg = xr.open_dataset(rgFilename, decode_times=False)
coll = create_collection(dbName='argo', collectionName='rgTempAnom')
coll.drop()
dataVariable='ARGO_TEMPERATURE_ANOMALY'
param='anomaly'
measurement='temperature'
gridName='rgTempAnom'
units = 'Degrees Celsius'

rgDataArray = rg[dataVariable]
insert_pres_time_grid(rgDataArray, coll, param, measurement, gridName, units,
                          dataVariable, insertOne=False)

2007-01-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
2007-02-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
2007-03-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
2007-04-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
2007-05-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
2007-06-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
2007-07-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
2007-08-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
2007-09-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
2007-10-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
2007-11-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom
2007-12-01 00:00:00
rgTempAnom
rgTempAnom
rgTempAnom


In [24]:
# make for express testing
testColl = create_collection(dbName='argo-express-test', collectionName='rgTempAnom')
#testColl.drop()
insert_pres_time_grid(rgDataArray, testColl, param, measurement, gridName, units,
                          dataVariable, insertOne=True)

2007-01-01 00:00:00
rgTempAnom
> <ipython-input-22-ba077474f814>(50)insert_pres_time_grid()
-> coll.insert_one(doc)
(Pdb) c


# Add mean field

In [5]:
rgFilename = '/home/tyler/Desktop/RG_ArgoClim_Temp.nc'
rg = xr.open_dataset(rgFilename, decode_times=False)
coll = create_collection(dbName='argo', collectionName='rgTempMean')
coll.drop()
dataVariable='ARGO_TEMPERATURE_MEAN'
param='mean'
measurement='temperature'
gridName='rgTemperatureAnomaly'
units = 'Degrees Celsius'

rgDataArray = rg[dataVariable]
insert_pres_time_grid(rgDataArray, coll, param, measurement, gridName, units,
                          dataVariable, insertOne=False)

NameError: name 'insert_pres_time_grid' is not defined

# Mask to matfile

Used for Kuusela-Stein gridding

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import pymongo
import pdb
from datetime import datetime, timedelta
from dateutil.relativedelta import *
from scipy.io import loadmat
import os
import glob

In [219]:
rgFilename = '/home/tyler/Desktop/RG_ArgoClim_Temp.nc'
rg = xr.open_dataset(rgFilename, decode_times=False)
rg.info

<bound method Dataset.info of <xarray.Dataset>
Dimensions:                   (LATITUDE: 145, LONGITUDE: 360, PRESSURE: 58, TIME: 181, bnds: 2)
Coordinates:
  * LONGITUDE                 (LONGITUDE) float32 20.5 21.5 22.5 ... 378.5 379.5
  * LATITUDE                  (LATITUDE) float32 -64.5 -63.5 -62.5 ... 78.5 79.5
  * PRESSURE                  (PRESSURE) float32 2.5 10.0 20.0 ... 1900.0 1975.0
  * TIME                      (TIME) float32 0.5 1.5 2.5 ... 178.5 179.5 180.5
Dimensions without coordinates: bnds
Data variables:
    PRESSURE_bnds             (PRESSURE, bnds) float32 ...
    ARGO_TEMPERATURE_ANOMALY  (TIME, PRESSURE, LATITUDE, LONGITUDE) float32 ...
    ARGO_TEMPERATURE_MEAN     (PRESSURE, LATITUDE, LONGITUDE) float32 ...
    BATHYMETRY_MASK           (PRESSURE, LATITUDE, LONGITUDE) float32 ...
    MAPPING_MASK              (PRESSURE, LATITUDE, LONGITUDE) float32 ...
Attributes:
    history:      FERRET V6.842    1-Feb-18FERRET V5.51   10-May-18FERRET V5....
    Conventions

In [235]:
mask = rg.BATHYMETRY_MASK.to_dataframe()
mask = mask.reset_index()
mask = mask[mask['PRESSURE'] == 10]
#mask.LONGITUDE = mask.LONGITUDE.apply(lambda long: long % 360)

In [236]:
mask.BATHYMETRY_MASK.unique()

array([ 1., nan,  0.])

In [237]:
print(mask.shape)
mask.head()

(52200, 4)


,PRESSURE,LATITUDE,LONGITUDE,BATHYMETRY_MASK
52200,10.0,-64.5,20.5,1.0
52201,10.0,-64.5,21.5,1.0
52202,10.0,-64.5,22.5,1.0
52203,10.0,-64.5,23.5,1.0
52204,10.0,-64.5,24.5,1.0


In [238]:
x = np.linspace(-89.5,89.5,180)
y = np.linspace(20.5,379.5,360)
xv, yv = np.meshgrid(x, y)
xv = np.reshape(xv, xv.shape[0] * xv.shape[1])
yv = np.reshape(yv, yv.shape[0] * yv.shape[1])

In [239]:
xv.shape

(64800,)

In [240]:
dfg = pd.DataFrame()
dfg['LATITUDE'] = xv
dfg['LONGITUDE'] = yv
dfg = dfg.sort_values(by=['LATITUDE', 'LONGITUDE'], ascending=[True, True])
dfg = dfg.reset_index()
dfg = dfg.drop('index', axis=1)

In [241]:
dfg.iloc[iCols, 0:3]

,LATITUDE,LONGITUDE
32853,1.5,113.5
32855,1.5,115.5
33198,2.5,98.5
33557,3.5,97.5
33559,3.5,99.5
33570,3.5,110.5
33571,3.5,111.5
33572,3.5,112.5
33573,3.5,113.5
33916,4.5,96.5


In [242]:
dfm = pd.merge(dfg, mask, how='left', on=['LATITUDE', 'LONGITUDE'])
dfm.head()

,LATITUDE,LONGITUDE,PRESSURE,BATHYMETRY_MASK
0,-89.5,20.5,NaN,NaN
1,-89.5,21.5,NaN,NaN
2,-89.5,22.5,NaN,NaN
3,-89.5,23.5,NaN,NaN
4,-89.5,24.5,NaN,NaN


In [243]:
dfm = dfm.fillna(0)
# masked values have a boolean true
dfm.BATHYMETRY_MASK = dfm.BATHYMETRY_MASK.apply(lambda x: int(x!=1))
mdict = dict()
mmask = dfm.BATHYMETRY_MASK.values
mmask = mmask.reshape((360, 180))
mdict['mask'] = dfm.BATHYMETRY_MASK.values

In [244]:
from scipy.io import savemat
savemat('/home/tyler/ArgoClim2019/mask.mat', mdict)

## Testing mask is correct

In [245]:
dfm[(dfm['LATITUDE']==1.5) & (dfm['LONGITUDE']==114.50)]

,LATITUDE,LONGITUDE,PRESSURE,BATHYMETRY_MASK
32854,1.5,114.5,10.0,1


In [247]:
iCols = [32853, 32855, 33198, 33557, 33559, 33570, 33571, 33572, 33573, 33916, 33927,
        33928, 33932, 33933, 33934, 34287, 34288, 34289, 34635, 34644, 35004, 35006,
        35355, 35356, 35367, 35700, 35714, 35715, 35716, 35725, 35728, 36060, 36074,
        36075, 36086, 36088, 36420, 36434, 36444, 36775, 36780, 36804, 37135, 37136,
        37494, 37495, 37496, 37499, 37524, 37854, 37855, 37856, 38214, 38215, 38216,
        38219, 38576, 38579, 38604, 38605, 38608, 38940, 38968, 39300, 32853, 32855]
iColsPy = [x - 1 for x in iCols] # python starts at index 0...
lats = [1.5000, 1.5000, 2.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 4.5000, 4.5000, 4.5000, 4.5000, 4.5000, 4.5000, 5.5000, 5.5000, 5.5000, 6.5000,
6.5000, 7.5000, 7.5000, 8.5000, 8.5000, 8.5000, 9.5000, 9.5000, 9.5000, 9.5000, 9.5000, 9.5000,10.5000,10.5000,10.5000,10.5000,10.5000,11.5000,11.5000,
11.5000,12.5000,12.5000,12.5000,13.5000,13.5000,14.5000,14.5000,14.5000,14.5000,14.5000,15.5000,15.5000,15.5000,16.5000,16.5000,16.5000,16.5000,
17.5000, 17.5000,17.5000,17.5000,17.5000,18.5000,18.5000,19.5000, 1.5000, 1.5000]
longs = [112.5000, 114.5000,  97.5000,  96.5000,  98.5000, 109.5000, 110.5000, 111.5000, 112.5000,  95.5000, 106.5000, 107.5000, 111.5000, 112.5000, 113.5000, 106.5000, 107.5000, 108.5000,  94.5000,
103.5000, 103.5000, 105.5000,  94.5000,  95.5000, 106.5000,  79.5000,  93.5000,  94.5000,  95.5000, 104.5000, 107.5000,  79.5000,  93.5000,  94.5000, 105.5000, 107.5000,  79.5000,  93.5000,
103.5000,  74.5000,  79.5000, 103.5000,  74.5000,  75.5000,  73.5000,  74.5000,  75.5000,  78.5000, 103.5000,  73.5000,  74.5000,  75.5000,  73.5000,  74.5000,  75.5000,  78.5000,  75.5000,
78.5000, 103.5000, 104.5000, 107.5000,  79.5000, 107.5000,  79.5000, 112.5000, 114.5000]

In [248]:
df = dfm.iloc[iColsPy, 0:4]


In [249]:
print(df.LATITUDE.values.tolist() == lats)
print(df.LONGITUDE.values.tolist() == longs)

True
True
